In [2]:
import pandas as pd
import numpy as np
import os
import io

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline 

In [3]:
import boto3
import sagemaker

In [4]:
s3_client = boto3.client('s3')
bucket_name = 'mvc-crashes'

In [15]:
obj_list=s3_client.list_objects(Bucket=bucket_name)

In [16]:
files=[]
for contents in obj_list['Contents']:
    files.append(contents['Key'])
    
print(files)

['mvc_crashes_part.csv']


In [14]:
file_name = 'mvc_crashes_part.csv' #files[0]
data_object = s3_client.get_object(Bucket=bucket_name, Key=file_name)
display(data_object)

{'ResponseMetadata': {'RequestId': '20592472A3934893',
  'HostId': 'KfjbWzE0OEFxYiJdh8dcKgWeU1FOT+GrZn0XMqH93CnSbp34RNYRWErXpLH9QV3MeeTo+W3ffAo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'KfjbWzE0OEFxYiJdh8dcKgWeU1FOT+GrZn0XMqH93CnSbp34RNYRWErXpLH9QV3MeeTo+W3ffAo=',
   'x-amz-request-id': '20592472A3934893',
   'date': 'Sun, 24 May 2020 23:23:10 GMT',
   'last-modified': 'Sun, 24 May 2020 23:19:41 GMT',
   'etag': '"4e35a27a45a6d3a6e105db43fce9fa6b-9"',
   'accept-ranges': 'bytes',
   'content-type': 'text/csv',
   'content-length': '144996778',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2020, 5, 24, 23, 19, 41, tzinfo=tzutc()),
 'ContentLength': 144996778,
 'ETag': '"4e35a27a45a6d3a6e105db43fce9fa6b-9"',
 'ContentType': 'text/csv',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f1f29fdd5f8>}

In [17]:
data_body = data_object["Body"].read()
print('Data type: ', type(data_body))

Data type:  <class 'bytes'>


In [19]:
data_stream = io.BytesIO(data_body)
crashes_df = pd.read_csv(data_stream, header=0, delimiter=",") 
crashes_df.head()

,crashes_datetime,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2018-06-20 17:25:00,NaN,NaN,40.805820,-73.909260,POINT (-73.90926 40.80582),BRUCKNER EXPRESSWAY,NaN,NaN,0.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,3925049,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
1,2018-07-09 09:20:00,NaN,NaN,40.842750,-73.924774,POINT (-73.924774 40.84275),WEST 171 STREET,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,3938002,Sedan,NaN,NaN,NaN,NaN
2,2018-06-28 12:06:00,BRONX,10453.0,40.847180,-73.921350,POINT (-73.92135 40.84718),MONTGOMERY AVENUE,WEST 174 STREET,NaN,0.0,...,Unspecified,NaN,NaN,NaN,3931268,Sedan,Taxi,NaN,NaN,NaN
3,2018-07-11 22:15:00,BROOKLYN,11237.0,40.698864,-73.909890,POINT (-73.90989 40.698864),NaN,NaN,454 WYCKOFF AVENUE,0.0,...,Unspecified,NaN,NaN,NaN,3941136,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2018-06-20 11:50:00,MANHATTAN,10016.0,40.748302,-73.978350,POINT (-73.97835 40.748302),EAST 37 STREET,LEXINGTON AVENUE,NaN,0.0,...,Unspecified,NaN,NaN,NaN,3925007,Sedan,Sedan,NaN,NaN,NaN


In [20]:
crashes_df.columns

Index(['crashes_datetime', 'borough', 'zip_code', 'latitude', 'longitude',
       'location', 'on_street_name', 'cross_street_name', 'off_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4',
       'contributing_factor_vehicle_5', 'collision_id', 'vehicle_type_code_1',
       'vehicle_type_code_2', 'vehicle_type_code_3', 'vehicle_type_code_4',
       'vehicle_type_code_5'],
      dtype='object')

In [21]:
# Bronx borough
crashes_df[crashes_df.borough=='BRONX'].head()

,crashes_datetime,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
2,2018-06-28 12:06:00,BRONX,10453.0,40.847180,-73.92135,POINT (-73.92135 40.84718),MONTGOMERY AVENUE,WEST 174 STREET,NaN,0.0,...,Unspecified,NaN,NaN,NaN,3931268,Sedan,Taxi,NaN,NaN,NaN
13,2018-06-19 17:15:00,BRONX,10455.0,40.815823,-73.91706,POINT (-73.91706 40.815823),NaN,NaN,421 EAST 149 STREET,0.0,...,Unspecified,NaN,NaN,NaN,3924341,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
20,2018-06-21 22:06:00,BRONX,10469.0,40.860893,-73.84321,POINT (-73.84321 40.860893),EASTCHESTER ROAD,WARING AVENUE,NaN,2.0,...,NaN,NaN,NaN,NaN,3932136,Sedan,NaN,NaN,NaN,NaN
21,2018-06-28 10:25:00,BRONX,10453.0,40.850857,-73.90765,POINT (-73.90765 40.850857),EAST TREMONT AVENUE,MORRIS AVENUE,NaN,0.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,3931265,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
23,2018-07-10 21:45:00,BRONX,10466.0,40.889550,-73.84524,POINT (-73.84524 40.88955),EAST 233 STREET,STRANG AVENUE,NaN,1.0,...,Unspecified,NaN,NaN,NaN,3939732,Pick-up Truck,Sedan,NaN,NaN,NaN


In [49]:
crashes_df.count()

crashes_datetime                 717295
borough                          457580
zip_code                         457440
latitude                         667480
longitude                        667480
location                         667480
on_street_name                   546426
cross_street_name                350378
off_street_name                  169677
number_of_persons_injured        717279
number_of_persons_killed         717265
number_of_pedestrians_injured    717295
number_of_pedestrians_killed     717295
number_of_cyclist_injured        717295
number_of_cyclist_killed         717295
number_of_motorist_injured       717295
number_of_motorist_killed        717295
contributing_factor_vehicle_1    715014
contributing_factor_vehicle_2    608045
contributing_factor_vehicle_3     46597
contributing_factor_vehicle_4      9920
contributing_factor_vehicle_5      2645
collision_id                     717295
vehicle_type_code_1              712848
vehicle_type_code_2              548336


In [68]:
# Rows with lat, lon set
crashes_df[~crashes_df['latitude'].isna()].head()

,crashes_datetime,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2018-06-20 17:25:00,NaN,NaN,40.805820,-73.909260,POINT (-73.90926 40.80582),BRUCKNER EXPRESSWAY,NaN,NaN,0.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,3925049,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
1,2018-07-09 09:20:00,NaN,NaN,40.842750,-73.924774,POINT (-73.924774 40.84275),WEST 171 STREET,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,3938002,Sedan,NaN,NaN,NaN,NaN
2,2018-06-28 12:06:00,BRONX,10453.0,40.847180,-73.921350,POINT (-73.92135 40.84718),MONTGOMERY AVENUE,WEST 174 STREET,NaN,0.0,...,Unspecified,NaN,NaN,NaN,3931268,Sedan,Taxi,NaN,NaN,NaN
3,2018-07-11 22:15:00,BROOKLYN,11237.0,40.698864,-73.909890,POINT (-73.90989 40.698864),NaN,NaN,454 WYCKOFF AVENUE,0.0,...,Unspecified,NaN,NaN,NaN,3941136,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2018-06-20 11:50:00,MANHATTAN,10016.0,40.748302,-73.978350,POINT (-73.97835 40.748302),EAST 37 STREET,LEXINGTON AVENUE,NaN,0.0,...,Unspecified,NaN,NaN,NaN,3925007,Sedan,Sedan,NaN,NaN,NaN


In [48]:
crashes_df[~crashes_df['latitude'].isna()].shape

(667480, 28)

In [31]:
crashes_df.iloc[1,0]  # cell

'2018-07-09 09:20:00'

In [35]:
# crashes_df.index
# Display only latitude and longitude
crashes_df[['latitude', 'longitude']].head()

,latitude,longitude
0,40.805820,-73.909260
1,40.842750,-73.924774
2,40.847180,-73.921350
3,40.698864,-73.909890
4,40.748302,-73.978350


In [36]:
# Count rows and columns
crashes_df.shape

(717295, 28)

In [37]:
# Number of killed people
crashes_df['number_of_persons_killed'].sum()

790.0

In [56]:
# Select data after 2019
crashes_df[crashes_df['crashes_datetime'] > '2018-12-04'].shape

(272373, 28)

In [55]:
# Select data after 2019 with lat, lon set
crashes_df[crashes_df['crashes_datetime'] > '2018-12-04'][~crashes_df['latitude'].isna()].shape

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


(250133, 28)

In [57]:
crashes_df.isna().sum()

crashes_datetime                      0
borough                          259715
zip_code                         259855
latitude                          49815
longitude                         49815
location                          49815
on_street_name                   170869
cross_street_name                366917
off_street_name                  547618
number_of_persons_injured            16
number_of_persons_killed             30
number_of_pedestrians_injured         0
number_of_pedestrians_killed          0
number_of_cyclist_injured             0
number_of_cyclist_killed              0
number_of_motorist_injured            0
number_of_motorist_killed             0
contributing_factor_vehicle_1      2281
contributing_factor_vehicle_2    109250
contributing_factor_vehicle_3    670698
contributing_factor_vehicle_4    707375
contributing_factor_vehicle_5    714650
collision_id                          0
vehicle_type_code_1                4447
vehicle_type_code_2              168959


In [73]:
columns = [
    'number_of_persons_injured', 'number_of_persons_killed',
    'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
    'number_of_cyclist_injured', 'number_of_cyclist_killed',
    'number_of_motorist_injured', 'number_of_motorist_killed']
clean_crashes_df = crashes_df[columns]
clean_crashes_df.loc[clean_crashes_df.number_of_persons_injured.isna(), 'number_of_persons_injured'] = 0
clean_crashes_df.loc[clean_crashes_df.number_of_persons_killed.isna(), 'number_of_persons_killed'] = 0
# clean_crashes_df.head()
clean_crashes_df.count()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


number_of_persons_injured        717295
number_of_persons_killed         717295
number_of_pedestrians_injured    717295
number_of_pedestrians_killed     717295
number_of_cyclist_injured        717295
number_of_cyclist_killed         717295
number_of_motorist_injured       717295
number_of_motorist_killed        717295
dtype: int64

In [59]:
from sagemaker import get_execution_role

session = sagemaker.Session() # store the current SageMaker session

# get IAM role
role = get_execution_role()
print(role)

arn:aws:iam::006275120779:role/service-role/AmazonSageMaker-ExecutionRole-20200401T204101


In [76]:
bucket_name = 'mvc-crashes'
prefix = 'output'
output_path = 's3://{}/{}'.format(bucket_name, prefix)

In [77]:
# define a PCA model
from sagemaker import PCA

# features - 1
N_COMPONENTS = len(clean_crashes_df.columns)-1

pca_SM = PCA(role=role,
             train_instance_count=1,
             train_instance_type='ml.c4.xlarge',
             output_path=output_path, # specified, above
             num_components=N_COMPONENTS, 
             sagemaker_session=session)

In [78]:
# convert df to np array
train_data_np = clean_crashes_df.values.astype('float32')

# convert to RecordSet format
formatted_train_data = pca_SM.record_set(train_data_np)

In [79]:
%%time

# train the PCA mode on the formatted data
pca_SM.fit(formatted_train_data)

2020-05-25 03:07:58 Starting - Starting the training job...
2020-05-25 03:08:00 Starting - Launching requested ML instances......
2020-05-25 03:09:09 Starting - Preparing the instances for training......
2020-05-25 03:10:23 Downloading - Downloading input data
2020-05-25 03:10:23 Training - Downloading the training image...
2020-05-25 03:10:54 Uploading - Uploading generated training model
2020-05-25 03:10:54 Completed - Training job completed
Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/25/2020 03:10:45 INFO 139699054421824] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'_num_gpus': u'auto', u'_log_level': u'info', u'subtract_mean': u'true', u'force_dense': u'true', u'epochs': 1, u'algorithm_mode': u'regular', u'extra_components': u'-1', u'_kvstore': u'dist_sync', u'_num_kv_servers': u'auto'}
[05/25/2020 03:10:45 INFO 139699054421824] Reading provided configur

Training seconds: 50
Billable seconds: 50
CPU times: user 422 ms, sys: 16.5 ms, total: 439 ms
Wall time: 3min 11s


In [80]:
training_job_name='pca-2020-05-25-03-07-58-435'

# where the model is saved, by default
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
print(model_key)

# download and unzip model
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')

# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

output/pca-2020-05-25-03-07-58-435/output/model.tar.gz


2304

In [1]:
import mxnet as mx

# loading the unzipped artifacts
pca_model_params = mx.ndarray.load('model_algo-1')

# what are the params
print(pca_model_params)

{'s': 
[  9.334156  12.114966  19.712225  35.957344 137.18034  241.06836
 757.8103  ]
<NDArray 7 @cpu(0)>, 'v': 
[[ 1.1148384e-03  4.9833992e-01  1.9023847e-04 -1.2229377e-03
   2.2062284e-01  4.1429052e-01 -7.2893262e-01]
 [-2.4137799e-01  1.6530571e-03 -7.2029948e-02  8.3013815e-01
  -1.4356098e-03  1.6615968e-03 -7.1781676e-04]
 [-1.0272411e-03 -5.0017762e-01 -1.6696181e-03 -2.2229475e-03
  -4.1156960e-01  7.6109856e-01 -3.3945121e-02]
 [ 3.3594552e-01 -2.0244371e-03  6.5687066e-01  4.5399353e-01
  -1.0840843e-03  1.8564900e-03 -1.5384601e-04]
 [-8.1912935e-04 -5.0322014e-01 -4.3285851e-04  1.4236759e-03
   8.5414428e-01  1.3067897e-01 -1.1242964e-02]
 [-8.5604441e-01  9.1218145e-04  6.3610757e-03  5.9289549e-02
   9.1741873e-05  2.5534775e-04 -1.4816609e-05]
 [-1.1770822e-03 -4.9823788e-01  4.1847551e-04  1.5992795e-04
  -2.2884516e-01 -4.8167300e-01 -6.8365031e-01]
 [ 3.0993691e-01 -6.7130086e-04 -7.5052577e-01  3.1817734e-01
  -4.5896813e-04 -4.6454152e-04 -5.4983696e-04]]
<NDArr